# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [2]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 300)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

In [4]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

# Create text_dict

In [9]:
import os

folder_path = '../text/2023-04-18'
encoding='ISO-8859-1'
all_files = []

for filename in os.listdir(folder_path):
    with open(os.path.join(folder_path, filename), 'r', encoding=encoding) as f:
        all_files.append(f.read())

text_dict = create_text_dict(all_files)
text_dict

{1: 'Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, an internal focus elicited superior hypertrophic increases in the elbow flexors compared to an external focus, but MT in the quadriceps was unaffected by attentional focus strategy. The differences in changes in elbow flexor size between INTERNAL and EXTERNAL (12.4% vs. 6.9%, respectively) translated into a large magnitude of effect favouring the INTERNAL condition (h2 p = 0.307). These findings partially support the common bodybuilding claim that a mind\x96muscle connection enhances muscle growth. Although we did not attempt to determine mechanistic reasons for discrepant findings between the upper and lower limbs, it can be speculated that subjects found it easier to focus on the elbow fl

## Create relevance_prompts_df

In [6]:
from itertools import product
prompts_dict = dict()
experiment_num = 1
relevance_prompts_dict = dict()

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

# prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
#     columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

summarize_task = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])

relevance_prompts_df = pd.DataFrame(product(task_part3, audience), 
    columns=['task part 3', 'audience'])



# prompts_df['prompt'] = prompts_df.apply(
#     lambda row: f"{row['prep_step']} {row['task part 1']}.", 
#     axis=1)
# prompts_df['relevance'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
relevance_prompts_df['relevance'] = relevance_prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
relevance_prompts_dict[experiment_num] = relevance_prompts_df
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


# Iteration 1

In [10]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def relevance(self, relevance_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                for index, choice in enumerate(response.choices):
                    simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                    self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                    self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        self.simple_summary_dict[simplify_iteration]['summary'] = self.summaries_dict
        self.simple_summary_dict[simplify_iteration]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1
n_choices = 2
qna_dict = dict()
chatbot_dict = dict()
simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

In [11]:
qna_dict[iteration_id]

,article_title,system_role,model,prep step,summarization task,summary,text
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study also found that attentional focus did not have a significant effect on lower extremity hypertrophy, p...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that an internal focus of attention is better than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not have an impact on lower extremity hypertrophy. The findings suggest that individuals wi...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Hey! I just read about a study that investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an...,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read a very interesting study about the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but there was no effect ...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"Weight stigma is associated with poorer health behaviors, such as disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI, according to a study by the Eating in America Study. Furthermore, individuals across the weight spectrum reported experiencing weight stigm...

In [22]:
chatbot_dict[iteration_id]['text1_prompt00'].summaries_dict

{'response_01': 'A new study has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study also found that attentional focus did not have a significant effect on lower extremity hypertrophy, possibly due to the difficulty of establishing a "mind-muscle connection" in the thigh musculature during resistive exercise for untrained individuals. Further research is needed to determine the effects of attentional focus on individuals with previous resistance training experience.',
 'response_02': 'New research shows that an internal focus of attention is better than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not have an impact on lower extremity hypertrophy. The findings suggest that individuals with resistance training experience may be able to enhance quadriceps hypertrophy by adop

In [12]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary giv

In [18]:
pd.DataFrame(simple_summaries_dict[iteration_id][0])

,1
simple_summary01_option01,"A new study found that when you focus on the muscles inside your body, it helps them grow bigger and stronger when you exercise for a long time. This works best for the muscles in your arms. But for the muscles in your legs, it's hard to focus on them and make them grow bigger. This might be bec..."
simple_summary01_option02,"A new study found that when you're lifting weights, it's better to focus on the muscles you're using inside your body rather than on things outside your body. This helps your arm muscles grow bigger. However, it's hard to do this with your leg muscles. The study didn't find a big difference for ..."
simple_summary02_option01,"Scientists found out that when you focus on the muscles inside your arm while lifting weights, it helps make them bigger and stronger. But when you focus on something outside of your body, like the weight you're lifting, it doesn't make as much of a difference. However, when it comes to your leg..."
simple_summary02_option02,"The way you think about your muscles while lifting weights can affect how much they grow. When you focus on the muscles you are using, they grow more in your arms. But this doesn't work as well for your legs. People who have been working out for a while might be able to make their leg muscles gr..."
simplify task,Use terms a 12-year-old can understand.
summary,{'response_01': 'A new study has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study also found that attentional focus did not have a significant effect on lower extremi...


In [16]:
print(len(simple_summaries_dict[iteration_id]))
print([value.keys() for value in simple_summaries_dict[iteration_id]])
simple_summaries_dict[iteration_id]

4
[dict_keys([1]), dict_keys([1]), dict_keys([1]), dict_keys([1])]


[{1: {'simple_summary01_option01': "A new study found that when you focus on the muscles inside your body, it helps them grow bigger and stronger when you exercise for a long time. This works best for the muscles in your arms. But for the muscles in your legs, it's hard to focus on them and make them grow bigger. This might be because it's difficult to think about your leg muscles while you exercise. More research is needed to understand how this works for people who have exercised a lot before.",
   'simple_summary01_option02': "A new study found that when you're lifting weights, it's better to focus on the muscles you're using inside your body rather than on things outside your body. This helps your arm muscles grow bigger. However, it's hard to do this with your leg muscles. The study didn't find a big difference for leg muscles. They need more research to find out if people who already lift weights will have the same results.",
   'simple_summary02_option01': "Scientists found out 

## 1.2

In [23]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                # for index, choice in enumerate(response.choices):
                #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                #     print(f'\t...Summary given')
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]] = [index, choice["message"]["content"]]                    
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        self.simple_summary_dict[simplify_iteration]['summary'] = self.summaries_dict
        self.simple_summary_dict[simplify_iteration]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.2
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

### qna_dict

In [24]:
qna_dict[iteration_id]

,article_title,system_role,model,prep step,summarization task,summary,text
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that a ""mind-muscle c...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that adopting an internal focus of attention during long-term resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not have a significant impact on lower extremity hypertrophy. The...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Hey! Have you heard about the latest study on attentional focus strategies during long-term resistance training? It's really fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus. This means...,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, did you know that a recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations? The study found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elbow flex...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that weight stigma is significantly associated with disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study, which involved a large sample of U.S. adults, suggests that weight stigma may be a barrier to healthy behaviors and cou..

In [25]:
chatbot_dict[iteration_id]['text1_prompt00'].summaries_dict

{'response_01': 'New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that a "mind-muscle connection" may enhance muscle growth in certain muscle groups. Future research is needed to determine the effects of attentional focus on trained individuals.',
 'response_02': 'A new study has found that adopting an internal focus of attention during long-term resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not have a significant impact on lower extremity hypertrophy. These findings suggest that cueing strategies can affect hypertrophic outcomes and may be of interest to bodybuilders and fitness enthusiasts. Further research is needed to determine the strength- and hypertrophy-rel

In [26]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.2
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary g

### Simple summaries

In [28]:
pd.DataFrame(simple_summaries_dict[iteration_id][0])

,1
"A new study has found that adopting an internal focus of attention during long-term resistance training can lead to superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not have a significant impact on lower extremity hypertrophy. These findings suggest that cueing strategies can affect hypertrophic outcomes and may be of interest to bodybuilders and fitness enthusiasts. Further research is needed to determine the strength- and hypertrophy-related effects of different attentional focus strategies on individuals with previous resistance training experience.","[1, A new study found that thinking about your muscles while lifting weights can make them bigger in your arms, but not necessarily in your legs. This means that the way you think while exercising can affect how your muscles grow. This information can be useful for people who like to work out an..."
"New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that a ""mind-muscle connection"" may enhance muscle growth in certain muscle groups. Future research is needed to determine the effects of attentional focus on trained individuals.","[1, Scientists did a study on how to make muscles bigger when doing exercises like lifting weights. They found that thinking about your muscles while you're doing the exercise can make your arm muscles grow more than if you think about something else. But this only works for your arm muscles, no..."
simplify task,Use terms a 12-year-old can understand.
summary,"{'response_01': 'New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that..."


In [29]:
pd.DataFrame(simple_summaries_dict[iteration_id][-1])

,1
"Hey! Did you know that weight stigma can affect our health behaviors? I just read a study that found that weight stigma was significantly associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, after controlling for covariates. The study also found that weight stigma is associated with poorer health behaviors, independent of BMI. It's concerning that multiple behaviors, for which individuals have some control over, may be undermined by weight stigma. The study also found that individuals across the weight spectrum, not only those with overweight or obese BMIs, reported weight stigma. It's interesting because weight stigma is prevalent among men too, and there is a lack of research on men's health outcomes related to weight stigma. One potential mechanism is stress. Individuals who experience greater stress may engage in more unhealthy coping behaviors. This study highlights weight stigma as a potential barrier to healthy behaviors, and suggests that one strategy to improve population health may be to reduce weight stigma.","[1, Did you know that people can be mean to others because of their weight? This can make people do things that aren't good for their health, like eating too much junk food or drinking too much alcohol. Even people who are not overweight can be affected by this meanness. It's important to know t..."
"Hey! I just read an interesting study about weight stigma and health behaviors. The researchers found that weight stigma is associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, but no such relationship was observed for physical activity. It's concerning because weight stigma may be undermining individuals' health behaviors, independent of their BMI. Also, the study found that individuals across the weight spectrum, not just those with overweight or obese BMIs, reported weight stigma. The study suggests that weight stigma is a potential barrier to healthy behaviors and highlights the need for weight-inclusive approaches to health promotion.","[1, Hey there! I just learned something interesting about how people feel about weight. Did you know that when people are made to feel bad about their weight, it can make them eat more unhealthy foods, drink more alcohol, and have trouble sleeping? This happens to everyone, not just people who a..."
simplify task,Use terms a 12-year-old can understand.
summary,"{'response_01': 'Hey! I just read an interesting study about weight stigma and health behaviors. The researchers found that weight stigma is associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, but no such relationship was observed for physical activity...."


In [35]:
pd.Series(simple_summaries_dict[iteration_id][-1])[1]

{"Hey! I just read an interesting study about weight stigma and health behaviors. The researchers found that weight stigma is associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, but no such relationship was observed for physical activity. It's concerning because weight stigma may be undermining individuals' health behaviors, independent of their BMI. Also, the study found that individuals across the weight spectrum, not just those with overweight or obese BMIs, reported weight stigma. The study suggests that weight stigma is a potential barrier to healthy behaviors and highlights the need for weight-inclusive approaches to health promotion.": [1,
  "Hey there! I just learned something interesting about how people feel about weight. Did you know that when people are made to feel bad about their weight, it can make them eat more unhealthy foods, drink more alcohol, and have trouble sleeping? This happens to everyone, not just people who are over

### process_simple_summaries

In [33]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    # simple_summaries = pd.concat(
    #     [pd.DataFrame(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
    #     )
    simple_summaries = pd.DataFrame(
        [pd.Series(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
        )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (4, 1)


,1
0,"{'New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that a ""mind-muscle..."
1,{'Hey! Have you heard about the latest study on attentional focus strategies during long-term resistance training? It's really fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus. This mea...
2,"{'New research has found that weight stigma is significantly associated with disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study, which involved a large sample of U.S. adults, suggests that weight stigma may be a barrier to healthy behaviors and c..."
3,"{'Hey! I just read an interesting study about weight stigma and health behaviors. The researchers found that weight stigma is associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance, but no such relationship was observed for physical activity. It's concernin..."


In [27]:
print(len(simple_summaries_dict[iteration_id]))
print([value.keys() for value in simple_summaries_dict[iteration_id]])
simple_summaries_dict[iteration_id]

4
[dict_keys([1]), dict_keys([1]), dict_keys([1]), dict_keys([1])]


[{1: {'New research shows that an internal focus of attention during long-term resistance training can result in superior hypertrophic increases in the elbow flexors compared to an external focus. However, attentional focus did not affect lower extremity hypertrophy. This suggests that a "mind-muscle connection" may enhance muscle growth in certain muscle groups. Future research is needed to determine the effects of attentional focus on trained individuals.': [1,
    "Scientists did a study on how to make muscles bigger when doing exercises like lifting weights. They found that thinking about your muscles while you're doing the exercise can make your arm muscles grow more than if you think about something else. But this only works for your arm muscles, not your leg muscles. They need to do more studies to see if this works for people who are already good at exercising."],
   'A new study has found that adopting an internal focus of attention during long-term resistance training can lea

## 1.3

In [36]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        # for key in summaries_keys:
        #     new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
        #     print(f'\t\t...Preparing to summarize {key}')
        #     simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
        #     try:
        #         response = self.gpt(simplify_prompt, n_choices)
        #     except Exception as error:
        #         exc_type, exc_obj, tb = sys.exc_info()
        #         f = tb.tb_frame
        #         lineno = tb.tb_lineno
        #         filename = f.f_code.co_filename
        #         print("An error occurred on line", lineno, "in", filename, ":", error)
        #         print('\t**API request failed for `.simplify()`**')
        #         return self.qna
        #     try:
        #         # for index, choice in enumerate(response.choices):
        #         #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
        #         #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
        #         #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
        #         #     print(f'\t...Summary given')
        #         for index, choice in enumerate(response.choices):
        #             self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]] = [index, choice["message"]["content"]]                    
        #             print(f'\t...Summary given')
        #     except Exception as error:
        #         exc_type, exc_obj, tb = sys.exc_info()
        #         f = tb.tb_frame
        #         lineno = tb.tb_lineno
        #         filename = f.f_code.co_filename
        #         print("An error occurred on line", lineno, "in", filename, ":", error)
        #         self.simple_summary_dict[simplify_iteration][new_key] = response
        #         print(f'\t...Error parsing response for summary request')
        #     print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
        #     time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        self.simple_summary_dict[simplify_iteration]['summary'] = self.summaries_dict
        self.simple_summary_dict[simplify_iteration]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.3
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

In [37]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.3
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
**text2_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
**text2_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']


## 1.4

In [38]:
simplify_prompts

['', 'Use terms a 12-year-old can understand.']

In [39]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                # for index, choice in enumerate(response.choices):
                #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                #     print(f'\t...Summary given')
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]] = {
                        'choice': index, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['summary'] = self.summaries_dict
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.4
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

In [40]:
qna_dict[iteration_id]

,article_title,system_role,model,prep step,summarization task,summary,text
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not significan...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty for unt...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, did you know there was a study conducted on the effects of attentional focus strategies on long-term muscular adaptations? It's pretty interesting. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external fo...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read an interesting study about the effects of different attentional focus strategies during long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle t...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A study examining the relationship between weight stigma and health behaviors in a large sample of U.S. adults found that weight stigma was significantly associated with disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study suggests that weight stig.

In [41]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.4
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turb

In [42]:
simple_summaries_dict[iteration_id]

[{1: {'A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not significantly affect lower extremity hypertrophy. The study highlights the importance of cueing strategies in achieving optimal hypertrophic outcomes during resistance training.': {'choice': 1,
    'simple summary': 'Scientists recently studied how paying attention to different things during weightlifting can affect how your muscles grow over time. They found that thinking about the muscles you\'re working (an "internal" focus) helps your arm muscles get bigger, but doesn\'t make much difference for your leg muscles. This shows that it\'s important to think about the right things when you\'re lifting weights if you want to get the most muscle growth.',
    'simplify task': 'Use terms a 12-year-old 

In [45]:
pd.DataFrame(simple_summaries_dict[iteration_id][0])

,1
"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not significantly affect lower extremity hypertrophy. The study highlights the importance of cueing strategies in achieving optimal hypertrophic outcomes during resistance training.","{'choice': 1, 'simple summary': 'Scientists recently studied how paying attention to different things during weightlifting can affect how your muscles grow over time. They found that thinking about the muscles you're working (an ""internal"" focus) helps your arm muscles get bigger, but doesn't ma..."
"New research shows that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty for untrained individuals to establish a ""mind-muscle connection"" in the thigh musculature during resistive exercise. The study also had several limitations, such as the moderate repetition range used in the exercise protocol and the lack of accurate reporting on dietary practices throughout the study.","{'choice': 1, 'simple summary': 'A new study found that when people lift weights to make their arm muscles bigger, it's better to focus on your muscles and how they feel inside your body, rather than focusing on the weights or other things outside your body. But this didn't make a big difference..."


### process_simple_summaries

In [44]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    simple_summaries = pd.concat(
        [pd.DataFrame(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
        )
    # simple_summaries = pd.DataFrame(
    #     [pd.Series(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
    #     )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (8, 1)


,1
"A recent study investigated the effects of attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors, but does not significantly affect lower extremity hypertrophy. The study highlights the importance of cueing strategies in achieving optimal hypertrophic outcomes during resistance training.","{'choice': 1, 'simple summary': 'Scientists recently studied how paying attention to different things during weightlifting can affect how your muscles grow over time. They found that thinking about the muscles you're working (an ""internal"" focus) helps your arm muscles get bigger, but doesn't ma..."
"New research shows that an internal focus of attention is superior to an external focus when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not affect lower extremity hypertrophy, possibly due to the difficulty for untrained individuals to establish a ""mind-muscle connection"" in the thigh musculature during resistive exercise. The study also had several limitations, such as the moderate repetition range used in the exercise protocol and the lack of accurate reporting on dietary practices throughout the study.","{'choice': 1, 'simple summary': 'A new study found that when people lift weights to make their arm muscles bigger, it's better to focus on your muscles and how they feel inside your body, rather than focusing on the weights or other things outside your body. But this didn't make a big difference..."
"Hey! I just read an interesting study about the effects of different attentional focus strategies during long-term resistance training. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but muscle thickness in the quadriceps was unaffected. This partially supports the claim that a mind-muscle connection enhances muscle growth. Interestingly, attentional focus had markedly different effects specific to the upper versus lower limbs. The study had some limitations, but overall, it provides strong evidence that cueing strategies affect hypertrophic outcomes.","{'choice': 1, 'simple summary': 'Hey there! I just read about a cool study that looked at how different ways of thinking about our muscles can affect how they grow when we exercise. The study found that when people focused on their muscles from the inside (like feeling the muscle contract), thei..."
"Hey, did you know there was a study conducted on the effects of attentional focus strategies on long-term muscular adaptations? It's pretty interesting. The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but there was no significant difference in muscle thickness in the quadriceps between the two focus strategies. It's pretty cool because it partially supports the claim that a mind-muscle connection enhances muscle growth. Also, attentional focus had markedly different effects specific to the upper versus lower limbs, with internal focus resulting in a 16.2% increase in isometric elbow flexion strength versus a 2.6% increase in external focus. However, external focus resulted in greater peak isometric knee extension strength compared to internal focus. Overall, the study provides evidence that cueing strategies affect hypertrophic outcomes and that attentional focus can impact muscle growth in specific areas.","{'choice': 1, 'simple summary': 'Hey, did you know that scientists did a study on how you think about your muscles when you exercise? They found that if you focus on your muscles working inside your body, your arm muscles will get bigger. But for your leg muscles, it doesn't really matter how yo..."
"A study examining the relationship betwe

## 1.5

In [46]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                # for index, choice in enumerate(response.choices):
                #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                #     print(f'\t...Summary given')
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration] = {
                        'choice': index, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['summary'] = self.summaries_dict
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.5
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

In [49]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.5
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turb

In [52]:
pd.DataFrame(simple_summaries_dict[iteration_id][0]).transpose()

,choice,original summary,simple summary,simplify task
1,1,"A new study has found that an internal focus of attention is better than an external focus for maximizing muscle growth in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study suggests that individuals wi...","A new study found that when you are lifting weights, thinking about the muscles you are using helps them grow better than thinking about things around you. This works really well for your arm muscles, but doesn't make much difference for your leg muscles. The study suggests that people who lift ...",Use terms a 12-year-old can understand.


In [56]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    simple_summaries = pd.concat(
        [pd.DataFrame(chatbot_dict[iteration_id][key].simple_summary_dict).transpose() for key in chatbot_dict[iteration_id].keys()]
        )
    # simple_summaries = pd.DataFrame(
    #     [pd.Series(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
    #     )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (4, 4)


,choice,original summary,simple summary,simplify task
1,1,"A new study has found that an internal focus of attention is better than an external focus for maximizing muscle growth in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study suggests that individuals wi...","A new study found that when you are lifting weights, thinking about the muscles you are using helps them grow better than thinking about things around you. This works really well for your arm muscles, but doesn't make much difference for your leg muscles. The study suggests that people who lift ...",Use terms a 12-year-old can understand.
1,1,"Hey, did you know that a recent study found that focusing on the mind-muscle connection during resistance training can actually enhance muscle growth? The study showed that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external foc...","Hey, did you know that when you exercise with weights, thinking about the muscles you're using can help them grow bigger? A study found that when people focused on their muscles while lifting weights for their arms, their muscles grew more than when they focused on something else. But when they ...",Use terms a 12-year-old can understand.
1,1,"A new study has found that weight stigma is significantly associated with several health behaviors, including disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study highlights weight stigma as a potential barrier to healthy behaviors and suggests tha...","A recent study found that people who are treated badly because of their weight are more likely to have unhealthy behaviors like eating too much, drinking alcohol, and having trouble sleeping. This happens even if they aren't actually overweight. The study suggests that if we want to help people ...",Use terms a 12-year-old can understand.
1,1,"Hey, did you know that weight stigma is associated with poorer health behaviors, independent of BMI? I just read an interesting study that found weight stigma was significantly associated with disordered eating, comfort eating, alcohol use, and sleep disturbance. It's concerning that multiple be...","Did you know that some people are treated badly because of their weight, even if they are not too big? This can make them feel bad and do things like eat too much, drink alcohol, or not sleep well. The study says that this is not good for their health. It also shows that everyone, not just bigge...",Use terms a 12-year-old can understand.


In [50]:
print(len(simple_summaries_dict[iteration_id]))
print([value.keys() for value in simple_summaries_dict[iteration_id]])
simple_summaries_dict[iteration_id]

4
[dict_keys([1]), dict_keys([1]), dict_keys([1]), dict_keys([1])]


[{1: {'choice': 1,
   'simple summary': "A new study found that when you are lifting weights, thinking about the muscles you are using helps them grow better than thinking about things around you. This works really well for your arm muscles, but doesn't make much difference for your leg muscles. The study suggests that people who lift weights a lot might be able to make their leg muscles grow better if they think about using their leg muscles while they lift weights. But we need more research to be sure.",
   'simplify task': 'Use terms a 12-year-old can understand.',
   'original summary': 'A new study has found that an internal focus of attention is better than an external focus for maximizing muscle growth in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study suggests that individuals with resistance training experience may be able to enhance quadriceps hypertrophy by adopting an internal f

## 1.6

In [57]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                # for index, choice in enumerate(response.choices):
                #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                #     print(f'\t...Summary given')
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[index] = {
                        'choice': index, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['summary'] = self.summaries_dict
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.6
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

### qna_dict

In [62]:
print(qna_dict[iteration_id].shape)
qna_dict[iteration_id]

(8, 7)


,article_title,system_role,model,prep step,summarization task,summary,text
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The results showed that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertrophy of the elb...,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New study finds that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide insights into how cuei...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Hey! I just read a really interesting study on the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus of attention is better than an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors. ...,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Hey! I just read a really interesting study about the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention (i.e. focusing on the muscles being worked) resulted in superior hypertrophic increases in the elbow flexors compare...,"Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research from the Eating in America Study shows that weight stigma is linked to poorer health behaviors, including disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study suggests that weight stigma may be a barrier to healthy behaviors and highli...","W

In [59]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.6
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turb

In [60]:
print(len(simple_summaries_dict[iteration_id]))
print([value.keys() for value in simple_summaries_dict[iteration_id]])
simple_summaries_dict[iteration_id]

4
[dict_keys([1, 0]), dict_keys([1, 0]), dict_keys([1, 0]), dict_keys([1, 0])]


[{1: {'choice': 1,
   'simple summary': 'A new study found that thinking about your muscles while doing certain exercises can make them grow better. This is especially true for the muscles in your arms, but not as much for the muscles in your legs. This information can help people make better workout plans. #fitness #exercise #muscles #attention',
   'simplify task': 'Use terms a 12-year-old can understand.',
   'original summary': 'New study finds that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide insights into how cueing strategies can affect hypertrophic outcomes and may be useful for optimizing resistance training programs. #fitness #resistancetraining #hypertrophy #attentionalfocus'},
  0: {'choice': 0,
   'simple summary': "A new study found that when you're working out your a

### closer

In [61]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    simple_summaries = pd.concat(
        [pd.DataFrame(chatbot_dict[iteration_id][key].simple_summary_dict).transpose() for key in chatbot_dict[iteration_id].keys()]
        )
    # simple_summaries = pd.DataFrame(
    #     [pd.Series(chatbot_dict[iteration_id][key].simple_summary_dict) for key in chatbot_dict[iteration_id].keys()]
    #     )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (8, 4)


,choice,simple summary,simplify task,original summary
1,1,"A new study found that thinking about your muscles while doing certain exercises can make them grow better. This is especially true for the muscles in your arms, but not as much for the muscles in your legs. This information can help people make better workout plans. #fitness #exercise #muscles ...",Use terms a 12-year-old can understand.,"New study finds that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide insights into how cuei..."
0,0,"A new study found that when you're working out your arm muscles, it's better to focus on your muscles and how they feel, rather than what's going on around you. This helps your muscles grow bigger over time. But when you're working out your leg muscles, what you focus on doesn't seem to make a b...",Use terms a 12-year-old can understand.,"New study finds that an internal focus of attention is superior to an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide insights into how cuei..."
1,1,"Hey! Did you know that when you exercise, where you focus your attention can make a big difference in how your muscles grow? A study found that when people focused on their muscles while lifting weights, their arm muscles got bigger than when they focused on their movements or form. This is why ...",Use terms a 12-year-old can understand.,Hey! I just read a really interesting study about the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention (i.e. focusing on the muscles being worked) resulted in superior hypertrophic increases in the elbow flexors compare...
0,0,"Hey there! I just read a really cool study about how thinking about your muscles while you exercise can make them grow bigger. The study found that when people focused on the muscles they were working, like their biceps, they got stronger and bigger muscles compared to when they just focused on ...",Use terms a 12-year-old can understand.,Hey! I just read a really interesting study about the effects of attentional focus strategies on long-term resistance training. The study found that an internal focus of attention (i.e. focusing on the muscles being worked) resulted in superior hypertrophic increases in the elbow flexors compare...
1,1,"A study found that people who are treated badly because of their weight are more likely to have unhealthy habits like eating too much, drinking too much alcohol, and having trouble sleeping. This is true even if their weight is not actually a health problem. But being treated badly because of we...",Use terms a 12-year-old can understand.,"Weight stigma is associated with poorer health behaviors, independent of BMI, according to a study that examined the relationship between weight stigma and several health behaviors in a large sample of US adults. The study found that weight stigma was significantly associated with greater disord..."
0,0,"When people are made to feel bad or ashamed about their weight, it can make them do things that aren't good for their health like eating too much or drinking too much alcohol. This study found that weight stigma, or being made to feel bad about your weight, can cause these unhealthy behaviors ev...",Use terms a 12-year-old can understand.,"Weight stigma is associated with poorer health behaviors, independent of BMI, according to a study that examined the relationship between weight stigma and several health behaviors in a large sample of US adults. The study found that weight stigma was significantly associated with greater disord..."
1,1,"Hey there! I read a really cool study about how people fe

## 1.7

In [66]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                # for index, choice in enumerate(response.choices):
                #     simple_summary_option = f'{new_key}_option{"{:02d}".format(index+1)}'
                #     self.simple_summary_dict[simplify_iteration].setdefault(simple_summary_option, {})
                #     self.simple_summary_dict[simplify_iteration][simple_summary_option] = choice["message"]["content"]                    
                #     print(f'\t...Summary given')
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'choice': index+1, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['summary'] = self.summaries_dict
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['simplify task'] = simplify_task
        return self.simple_summary_dict


def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')

    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')
        # simple_summaries_master_dict[text_prompt_key] = dict()

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


iteration_id = 1.7
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

### qna_dict

In [67]:
print(qna_dict[iteration_id].shape)
qna_dict[iteration_id]

(8, 7)


,article_title,system_role,model,prep step,summarization task,summary,text
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention for maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide valuable insig...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about this new study on attentional focus strategies during resistance training? It's pretty fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but interestingly, mus...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the new study that investigated the effects of different attentional focus strategies on long-term muscular adaptations? Apparently, the study found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertroph...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that weight stigma is associated with poorer health behaviors, including disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study also found that targeting health behaviors to achieve weight loss, assuming that individuals can improv.

In [68]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.7
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turb

In [69]:
print(len(simple_summaries_dict[iteration_id]))
print([value.keys() for value in simple_summaries_dict[iteration_id]])
simple_summaries_dict[iteration_id]

4
[dict_keys([1]), dict_keys([1]), dict_keys([1]), dict_keys([1])]


[{1: {'response_01': {0: {'choice': 1,
     'simple summary': "Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This was especially true for the muscles in your arms, but it didn't make a big difference for the muscles in your legs. These findings can help people make their workouts better and get stronger faster.",
     'simplify task': 'Use terms a 12-year-old can understand.',
     'original summary': 'A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentional focus did not affect lower extremity hypertrophy. T

In [83]:
print(simple_summaries_dict[iteration_id][0].keys())
simple_summaries_dict[iteration_id][0][1]

dict_keys([1])


{'response_01': {0: {'choice': 1,
   'simple summary': "Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This was especially true for the muscles in your arms, but it didn't make a big difference for the muscles in your legs. These findings can help people make their workouts better and get stronger faster.",
   'simplify task': 'Use terms a 12-year-old can understand.',
   'original summary': 'A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentional focus did not affect lower extremity hypertrophy. These result

In [89]:
pd.DataFrame(simple_summaries_dict[iteration_id][0][1]['response_01']).transpose()

,choice,simple summary,simplify task,original summary
0,1,Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This wa...,Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
1,2,"Scientists recently did a study to see how people should think about their muscles when they exercise. They found that it's better to think about your muscles inside your body instead of outside of your body when working on your arm muscles. But for leg muscles, it doesn't really matter how you ...",Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."


In [95]:
chatbot_dict[iteration_id]['text1_prompt00'].simple_summary_dict

{1: {'response_01': {0: {'choice': 1,
    'simple summary': "Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This was especially true for the muscles in your arms, but it didn't make a big difference for the muscles in your legs. These findings can help people make their workouts better and get stronger faster.",
    'simplify task': 'Use terms a 12-year-old can understand.',
    'original summary': 'A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentional focus did not affect lower extremity hypertrophy. These

In [101]:
pd.DataFrame(chatbot_dict[iteration_id]['text1_prompt00'].simple_summary_dict[1]['response_01']).transpose()

,choice,simple summary,simplify task,original summary
0,1,Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This wa...,Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
1,2,"Scientists recently did a study to see how people should think about their muscles when they exercise. They found that it's better to think about your muscles inside your body instead of outside of your body when working on your arm muscles. But for leg muscles, it doesn't really matter how you ...",Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."


In [76]:
pd.concat([pd.DataFrame(value) for value in simple_summaries_dict[iteration_id][0].values()])

,response_01,response_02
0,"{'choice': 1, 'simple summary': 'Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's hap...","{'choice': 1, 'simple summary': 'A new study shows that when you're lifting weights, it's better to focus on the muscles you're using inside your body instead of what's happening outside of your body. This helps your arm muscles get bigger. But it doesn't really matter where you focus your atten..."
1,"{'choice': 2, 'simple summary': 'Scientists recently did a study to see how people should think about their muscles when they exercise. They found that it's better to think about your muscles inside your body instead of outside of your body when working on your arm muscles. But for leg muscles, ...","{'choice': 2, 'simple summary': 'A new research study shows that if you want to make your arm muscles bigger when you lift weights, it’s better to think about your muscles and how they feel inside your body, rather than focusing on things outside of your body. But when it comes to making your le..."


## process_simple_summaries

In [103]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    list = []
    for chatbot_key in chatbot_dict[iteration_id].keys():
        # pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[1]['response_01']).transpose()
        # print(chatbot_key)
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            # print(simplify_iteration_key)
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                # print(response_key)
                # print(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key])
                list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    
    simple_summaries = pd.concat(list)
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (16, 4)


,choice,simple summary,simplify task,original summary
0,1,Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This wa...,Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
1,2,"Scientists recently did a study to see how people should think about their muscles when they exercise. They found that it's better to think about your muscles inside your body instead of outside of your body when working on your arm muscles. But for leg muscles, it doesn't really matter how you ...",Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
0,1,"A new study shows that when you're lifting weights, it's better to focus on the muscles you're using inside your body instead of what's happening outside of your body. This helps your arm muscles get bigger. But it doesn't really matter where you focus your attention when you're working on your ...",Use terms a 12-year-old can understand.,"A new study has found that an internal focus of attention is superior to an external focus of attention for maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide valuable insig..."
1,2,"A new research study shows that if you want to make your arm muscles bigger when you lift weights, it’s better to think about your muscles and how they feel inside your body, rather than focusing on things outside of your body. But when it comes to making your leg muscles bigger, it doesn’t real...",Use terms a 12-year-old can understand.,"A new study has found that an internal focus of attention is superior to an external focus of attention for maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide valuable insig..."
0,1,"Have you heard about a cool new study on how we focus our attention during exercise? It found that thinking about the muscles we're using (internal focus) can help them grow bigger in our arms, but it didn't really make a difference for our leg muscles. This supports the idea that thinking about...",Use terms a 12-year-old can understand.,"Hey, have you heard about this new study on attentional focus strategies during resistance training? It's pretty fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but interestingly, mus..."
1,2,"Have you heard about a new study on how to focus your mind when doing weight exercises? The study found that thinking about the muscle you're working on (internal focus) can help it grow bigger, but thinking about the movement you're doing (external focus) doesn't make a difference for some musc...",Use terms a 12-year-old can understand.,"Hey, have you heard about this new study on attentional focus strategies during resistance training? It's pretty fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but interestingly, mus..."
0,1,Have you heard about the new study on how we can grow ou

### Got it

In [107]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    list = []
    for chatbot_key in chatbot_dict[iteration_id].keys():
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    
    simple_summaries = pd.concat(list)
    # simple_summaries = simple_summaries.merge(
    #     qna_dict[iteration_id], how='left', left_on='original summary',
    #     right_on='summary'
    #     )
    simple_summaries = qna_dict[iteration_id].merge(
        simple_summaries, how='right', right_on='original summary',
        left_on='summary'
        )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (16, 11)


,article_title,system_role,model,prep step,summarization task,summary,text,choice,simple summary,simplify task,original summary
0,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,Researchers recently looked at how different ways of thinking about the muscles you're working affect how much they grow when you do strength exercises. They found that focusing on how your muscles feel inside your body works better than thinking about what's happening outside your body. This wa...,Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",2,"Scientists recently did a study to see how people should think about their muscles when they exercise. They found that it's better to think about your muscles inside your body instead of outside of your body when working on your arm muscles. But for leg muscles, it doesn't really matter how you ...",Use terms a 12-year-old can understand.,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio..."
2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention for maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide valuable insig...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,"A new study shows that when you're lifting weights, it's better to focus on the 

In [109]:
qna_dict[iteration_id].name = 'choice'

In [111]:
qna_dict[iteration_id].reset_index(names = ['choice'])

,choice,article_title,system_role,model,prep step,summarization task,summary,text
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A recent study investigated the effects of different attentional focus strategies on long-term muscular adaptations during resistance training. The findings suggest that an internal focus of attention is superior to an external focus in maximizing hypertrophy of the elbow flexors, while attentio...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention for maximizing hypertrophy of the elbow flexors during resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide valuable insig...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about this new study on attentional focus strategies during resistance training? It's pretty fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus, but interestingly, mus...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
3,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the new study that investigated the effects of different attentional focus strategies on long-term muscular adaptations? Apparently, the study found that an internal focus of attention is superior to an external focus of attention when the goal is to maximize hypertroph...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
4,1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that weight stigma is associated with poorer health behaviors, including disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study also found that targeting health behaviors to achieve weight loss, assuming that indivi

## 1.8 Works for adding simple summaries

In [117]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, max_tokens=1000, 
                    simplify_iteration=None, pause_per_request=20
                    ):
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
            time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['summary'] = self.summaries_dict
        # self.simple_summary_dict[simplify_iteration][self.summaries_dict[key]]['simplify task'] = simplify_task
        return self.simple_summary_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    prompt_column='prompt', pause_per_request=20,
    save_outputs=False, filename=None, append_version=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.

    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(task=task, prep_step=prep_step, n_choices=n_choices)
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=20,
    prompt_column='simplify', simplify_iteration=None, save_outputs=False,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index, prompt in enumerate(simplify_prompts):
            relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    list = []
    for chatbot_key in chatbot_dict[iteration_id].keys():
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    simple_summaries = pd.concat(list)
    simple_summaries = qna_dict[iteration_id].merge(
        simple_summaries, how='right', right_on='original summary',
        left_on='summary'
        )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries

iteration_id = 1.8
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
#     n_choices=2, pause_per_request=pause_per_request,
#     simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
#     )


# process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

### qna_dict

In [118]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,prep step,summarization task,summary,text
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found no significant effect on lower extremity hypertrophy, which may be due to difficulty establish...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about this study on attentional focus during resistance training? It found that an internal focus, where people focus on the muscle being worked, led to greater hypertrophy in the elbow flexors compared to an external focus, where people focus on the movement itself. This sup...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
3,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the latest study on attentional focus strategies during resistance training? It's the first one to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus of attention is superior to an ...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
4,1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that weight stigma is linked to poorer health behaviors, such as disordered eating, comfort eating, alcohol use, and sleep disturbance, independent of BMI. The study also found that weight stigma affects individuals across the weight spectrum, not just those w

In [119]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_dict[summary_iteration_id], iteration_id,
    n_choices=2, pause_per_request=pause_per_request,
    simplify_iteration=1, summary_iteration_id=summary_iteration_id, save_outputs=False
    )

summary_iteration_id: 1.8
n_choices: 2
**text1_prompt00
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
**text1_prompt01
	...Prompt: Use terms a 12-year-old can understand., iteration 1
simplify_iteration:  1
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turb

In [201]:
simple_summaries_dict[iteration_id]

[{1: {'response_01': {0: {'simple summary choice': 1,
     'simple summary': "A new study found that when you focus on the muscle you're working on while lifting weights, your muscles grow bigger and stronger. But this only works for your arms, not for your legs. This means that paying attention to your muscles while lifting weights can help you get better results in the long run.",
     'simplify task': 'Use terms a 12-year-old can understand.',
     'original summary': '"New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that cueing strategies affect muscular adaptations during long-term resistance training." #ScienceCommunication #ResistanceTraining #MuscularAdaptations'},
    1: {'simple summary choice': 2,
     'simple summary': "A recent study 

### Success with adding simple summaries to dataframe

In [120]:
qna_dict2 = dict()
qna_dict2[iteration_id] = process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (16, 12)


In [121]:
qna_dict2[iteration_id]

,choice,article_title,system_role,model,prep step,summarization task,summary,text,simple summary choice,simple summary,simplify task,original summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,"A new study found that when you focus on the muscle you're working on while lifting weights, your muscles grow bigger and stronger. But this only works for your arms, not for your legs. This means that paying attention to your muscles while lifting weights can help you get better results in the ...",Use terms a 12-year-old can understand.,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that..."
1,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",2,"A recent study found that when people focus on their muscles while lifting weights for their arms, they get stronger and bigger muscles compared to when they focus on the weight or the movement. But this doesn't seem to make a difference for leg muscles. This study shows that how we think about ...",Use terms a 12-year-old can understand.,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that..."
2,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found no significant effect on lower extremity hypertrophy, which may be due to difficulty establish...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,"Scientists found that when you focus on your m

### save outputs

In [122]:
save_output(qna_dict2[iteration_id], description='prompt_chain_experiment')

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/prompt_chain_experiment_2023-04-18_1549.sav
Time completed: 2023-04-18 15:49:38.968255
	Object saved as pickle
File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/CSV/prompt_chain_experiment_2023-04-18_1549.csv
Time completed: 2023-04-18 15:49:39.205727
	DataFrame saved as CSV


In [123]:
save_output(chatbot_dict[iteration_id], description='chatbot', csv_path=None)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-18_1550.sav
Time completed: 2023-04-18 15:50:13.140058
	Object saved as pickle


TypeError: unsupported operand type(s) for &: 'type' and 'bool'

In [124]:
def save_output2(df, filename=None, description=None, append_version=True, iteration_id=None,
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\CSV',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Save an Python object as both pickle and CSV. Automatically create filename using date and time 
    if not provided.
    
    """
    if description:
        filename = f'{description}_{datetime.now().strftime("%Y-%m-%d_%H%M")}'
        append_version = False
    elif filename == None:
        filename = f'{datetime.now().strftime("%Y-%m-%d_%H%M")}_outputs'
        append_version = False
    if iteration_id:
        filename += f'_{"{:02d}".format(iteration_id)}'
    try:
        savepickle(df, filename=filename, path=pickle_path, append_version=append_version)
        print('\tObject saved as pickle')
    except:
        print('Unable to save pickle')
    if type(df) == pd.core.frame.DataFrame:
        save_csv(df, filename=filename, path=csv_path, append_version=append_version)
        print('\tDataFrame saved as CSV')
    elif (type(df) == dict) & (csv_path != None):
        try:
            save_csv(pd.DataFrame(df), filename=filename, path=csv_path, append_version=append_version)
            print('\tDictionary converted to CSV')
        except:
            print('\tUnable to save CSV')

save_output2(chatbot_dict[iteration_id], description='chatbot', csv_path=None)

File saved:  C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/output/pickles/chatbot_2023-04-18_1551.sav
Time completed: 2023-04-18 15:51:24.121249
	Object saved as pickle


### Update process_simple_summaries

In [140]:
qna_dict[iteration_id].columns

Index(['choice', 'article_title', 'system_role', 'model', 'prep step',
       'summarization task', 'summary', 'text'],
      dtype='object')

In [144]:
qna_dict2[iteration_id][(qna_dict[iteration_id].columns)]

,choice,article_title,system_role,model,prep step,summarization task,summary,text
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found no significant effect on lower extremity hypertrophy, which may be due to difficulty establish...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
3,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found no significant effect on lower extremity hypertrophy, which may be due to difficulty establish...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
4,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about this study on attentional focus during resistance training? It found that an internal focus, where people focus on the muscle being worked, led to greater hypertrophy in the elbow flexors compared to an external focus, where people focus on the movement itself. This sup...","Dif

In [136]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame
    """
    list = []
    groupby_columns = qna_dict[iteration_id].columns
    for chatbot_key in chatbot_dict[iteration_id].keys():
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    simple_summaries = pd.concat(list)
    simple_summaries = qna_dict[iteration_id].merge(
        simple_summaries, how='right', right_on='original summary',
        left_on='summary'
        )
    print('simple_summaries.shape:', simple_summaries.shape)
    return simple_summaries.groupby(groupby_columns)
process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

simple_summaries.shape: (16, 12)


ValueError: Grouper and axis must be same length

In [188]:
qna_dict[iteration_id].columns

Index(['choice', 'article_title', 'system_role', 'model', 'prep step',
       'summarization task', 'summary', 'text'],
      dtype='object')

#### iteration 2

In [199]:
def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    
    # Get the groupby columns from qna_dict
    groupby_columns = qna_dict[iteration_id].columns.tolist()
    
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                
                # Append the dataframes to the list
                df_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    simple_summaries = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    simple_summaries = qna_dict[iteration_id].merge(
        simple_summaries, how='right', right_on='original summary', left_on='summary'
    )
    print('simple_summaries.shape:', simple_summaries.shape)
    
    return simple_summaries

process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

['choice', 'article_title', 'system_role', 'model', 'prep step', 'summarization task', 'summary', 'text']
simple_summaries.shape: (16, 12)


,choice,article_title,system_role,model,prep step,summarization task,summary,text,simple summary choice,simple summary,simplify task,original summary
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,"A new study found that when you focus on the muscle you're working on while lifting weights, your muscles grow bigger and stronger. But this only works for your arms, not for your legs. This means that paying attention to your muscles while lifting weights can help you get better results in the ...",Use terms a 12-year-old can understand.,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that..."
1,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",2,"A recent study found that when people focus on their muscles while lifting weights for their arms, they get stronger and bigger muscles compared to when they focus on the weight or the movement. But this doesn't seem to make a difference for leg muscles. This study shows that how we think about ...",Use terms a 12-year-old can understand.,"""New study reveals that an internal focus of attention during resistance training results in superior hypertrophic increases in elbow flexors compared to an external focus. However, attentional focus does not seem to affect lower extremity hypertrophy. These findings provide strong evidence that..."
2,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research has found that an internal focus of attention is more effective than an external focus for maximizing hypertrophy of the elbow flexors during long-term resistance training. The study found no significant effect on lower extremity hypertrophy, which may be due to difficulty establish...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...",1,"Scientists found that when you focus on your m

# Update simplify prompts

In [200]:
from itertools import product
prompts_dict = dict()
experiment_num = 1
relevance_prompts_dict = dict()

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]
task_part2 = [
    # "Use language for a lay audience",
    "Use terms a 12-year-old can understand.",
    "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

# prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
#     columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

summarize_task = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])

relevance_prompts_df = pd.DataFrame(product(task_part3, audience), 
    columns=['task part 3', 'audience'])



# prompts_df['prompt'] = prompts_df.apply(
#     lambda row: f"{row['prep_step']} {row['task part 1']}.", 
#     axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
relevance_prompts_df['relevance'] = relevance_prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
relevance_prompts_dict[experiment_num] = relevance_prompts_df
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


# iteration 2
* Modify iteration_id based on what is already present
* modify add_relevance
* See how to delete duplicate original summaries.

In [204]:
import time
class Chaining:
    """
    Requrired paramaters:
        - text (str): Text to feed to GPT for summarization.

    Optional parameters
        - system_role (str): ChatGPT parameter. 
            Default is "You are an expert at science communication."
        - temperature (float): ChatGPT parameter. Default is 0.7.
        - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
        - max_tokens (int): Token limit for ChatGPT response.
        - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, model="gpt-3.5-turbo", temperature=0.7, max_tokens=1000, 
        system_role = "You are an expert at science communication."):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task, text):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature):
        print('\tSending request to GPT-3')
        print(f'\t\tRequesting {n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, n_choices=5
        ):
        """
        SH 2023-04-11 12:18: Same as the user-defined `reply` function, but re-written as a class method.
        Send a ChatCompletion request to ChatGPT via the Chaining class.

        Requrired paramaters:
            - task (str): Task to include in ChatGPT prompt.
            - text (str): Text to feed to GPT for summarization.

        Optional parameters
            - system_role (str): ChatGPT parameter. 
                Default is "You are an expert at science communication."
            - temperature (float): ChatGPT parameter. Default is 0.7.
            - n_choices (int): Number of ChatGPT responses to generate. Default is 5.
            - max_tokens (int): Token limit for ChatGPT response.
            - model (str): ChatGPT model to use. Default is "gpt-3.5-turbo".
        """
        chatbot = Chaining(self.text)
        prompt = chatbot.create_prompt(task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()

        try:
            response = chatbot.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            self.qna[f'text'] = self.text
            self.summaries_dict['prep_step'] = prep_step
            self.summaries_dict['prompt'] = task
            return self.qna
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')
            return self.qna


    def simplify(self, simplify_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    # simplify_iteration=None, 
                    pause_per_request=20
                    ):
        simplify_iteration = len(self.summaries_dict) + 1 
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        print('simplify_iteration: ', simplify_iteration)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simple summary': choice["message"]["content"],
                        'simplify task': simplify_task,
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, system_role='You are an expert at science communication.',
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=20
                    ):
        relevance_iteration = len(self.summaries_dict) + 1 
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        print('relevance_iteration: ', relevance_iteration)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance statement': choice["message"]["content"],
                        'relevance task': relevance_task,
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance statement given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chaining_bot_dict, iteration_id, n_choices=5,
    temperature=0.7, pause_per_request=0,
    save_outputs=False, filename=None, 
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
    pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - prep_step, summarize_task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict
        - prompt_column (str): Name of the column in the prompts_df containing the user input.
    """
    temp_qna_dict = dict()
    qna_dfs_list = []
    prompts_df = pd.DataFrame(product(prep_step, summarize_task), 
        columns=['prep_step', 'summarize_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        temp_qna_dict[key] = dict()
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index} of {prompts_df.index.max()}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(text)
                print('Chaining class instance created')
                temp_qna_dict[key][index] = chatbot.summarize(
                    task=task, prep_step=prep_step, n_choices=n_choices,
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t...Error making chatbot request')
                break
        
        try:
            updated_qna_dict = (temp_qna_dict[key])
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('Error concatenating prompts DataFrame')
            return temp_qna_dict, chaining_bot_dict
        qna_dfs_list.append(updated_qna_dict)
    try:
        qna_dict[iteration_id] = pd.concat([
            pd.DataFrame(
                data, index=[choice for choice in range(1, len(data['summary'])+1)]
            ) for dictionary in qna_dfs_list for data in dictionary.values()
        ])
        qna_dict[iteration_id].reset_index(inplace=True, names=['choice'])
        print('DataFrame shape:', qna_dict[iteration_id].shape)
        if save_outputs:
            try:
                save_output(
                    qna_dict[iteration_id], description='prompt_chain_experiment',
                    csv_path=csv_path, pickle_path=pickle_path)
                save_output(
                    chaining_bot_dict[iteration_id], description='chaining_bot_dict',
                    csv_path=None, pickle_path=pickle_path)
            except:
                print('[prompt_chaining_dict()] Unable to save outputs')
    except Exception as error:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        print("An error occurred on line", lineno, "in", filename, ":", error)
        qna_dict[iteration_id] = qna_dfs_list
        print('Error creating DataFrame; dictionary returned instead')
    return qna_dict, chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', 
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)
    if type(simplify_prompts) == str:
        simplify_prompts = [simplify_prompts]
    elif type(simplify_prompts) == pd.core.frame.DataFrame:
        simplify_prompts = [prompt for prompt in simplify_prompts[prompt_column].unique() if len(prompt) >1]
    else:
        simplify_prompts = list(set([prompt for prompt in simplify_prompts if len(prompt) >1]))
    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index, prompt in enumerate(simplify_prompts):
            # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
            # print(f'\t...Prompt: {prompt}, iteration {relevance_iteration}')
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # simplify_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, n_choices=n_choices, pause_per_request=pause_per_request, 
                    # relevance_iteration=index if len(simplify_prompts) > 1 else simplify_iteration
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict


def process_simple_summaries(qna_dict, chatbot_dict, iteration_id):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        for simplify_iteration_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict.keys():
            for response_key in chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key].keys():
                df_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[simplify_iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    simple_summaries = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    simple_summaries = qna_dict[iteration_id].merge(
        simple_summaries, how='right', right_on='original summary', left_on='summary'
    )
    print('simple_summaries.shape:', simple_summaries.shape)
    
    return simple_summaries


iteration_id = 2
n_choices = 2
# qna_dict = dict()
# chatbot_dict = dict()
# simple_summaries_dict = dict()

summarize_task = task_part1
simplify_prompts = task_part2

summary_iteration_id = iteration_id
# summary_iteration_id = 3.9
pause_per_request=0
# relevance_prompts = relevance_prompts_df

qna_dict, chaining_dict = batch_summarize_chain(text_dict, prep_step, summarize_task, qna_dict, chatbot_dict, 
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id)

# simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    # chaining_dict[iteration_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )


# process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

# relevance_dict = prompt_chaining_dict(simplify_prompts, relevance_dict, 
#     chaining_dict[iteration_id], iteration_id,
#     prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
#     )

**Text #1 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #1 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #0 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating prompt
	Sending request to GPT-3
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Success!
[batch_summarize()] Sleeping 0 sec to avoid exceeding API rate limit
**Text #2 prompt #1 of 1**
Creating Chaining class instance
Chaining class instance created
	Done creating promp

### qna_dict

In [205]:
qna_dict[iteration_id]

,choice,article_title,system_role,model,prep step,summarization task,summary,text
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights in...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study also...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the latest study on attentional focus strategies during long-term resistance training? It's really fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus. This parti...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
3,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey! I just read about a really interesting study on the effects of different attentional focus strategies on long-term muscular adaptations. The study found that an internal focus of attention is actually superior to an external focus for maximizing hypertrophy of the elbow flexors, but attenti...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ..."
4,1,Weight stigma and health behaviors: evidence from the Eating in America Study,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"New research shows that weight stigma is associated with poorer health behaviors, independent of BMI. The study found that weight stigma was significantly associated with greater disordered eating, comfort eating, alcohol use, and sleep disturbance. These findings highlight weig

In [208]:
simple_summaries_dict = prompt_chaining_dict(simplify_prompts, simple_summaries_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )
qna_dict2[iteration_id] = process_simple_summaries(qna_dict, chatbot_dict, iteration_id)

summary_iteration_id: 2
n_choices: 1
**text1_prompt00
simplify_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
simplify_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Summary given
[.simplify()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to summarize response_02
	Done creating prompt
	Sending 

### Summaries + Simple Summaries

In [209]:
qna_dict2[iteration_id]

,choice,article_title,system_role,model,prep step,summarization task,summary,text,original summary,simple summary,simple summary choice,simplify task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights in...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","""New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights in...","A new study found that when you focus on the muscles you're working on while lifting weights, it can help them grow bigger. But this only seems to work for the muscles in your arms, not your legs. This tells us that our brain can help our muscles get stronger. But we need more research to know i...",1,Use terms a 12-year-old can understand.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study also...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A new study has found that an internal focus of attention is superior to an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study also...","A new study found that when you're lifting weights, it's better to focus on the muscles you're using inside your body rather than the things around you. This helps your arm muscles get bigger, but it doesn't seem to make a difference for your leg muscles. The study also showed that there's more ...",1,Use terms a 12-year-old can understand.
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the latest study on attentional focus strategies during long-term resistance training? It's really fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus. This parti...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","Hey, have y

In [210]:
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


In [212]:
relevance_dict = dict()
relevance_dict = prompt_chaining_dict(simplify_prompts, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id,
    prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 2
n_choices: 1
**text1_prompt00
relevance_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate limit
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate limit
relevance_iteration:  5
summaries_keys: 
	 ['response_01', 'response_02']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
[.add_relevance()] Sleeping 0 sec to avoid exceeding API rate lim

In [217]:
relevance_dict[iteration_id]

[{5: {'response_01': {0: {'relevance choice': 1,
     'relevance statement': "A new study found that when you focus on the muscles you're working on while lifting weights, it can help them grow bigger. But this only seems to work for the muscles in your arms, not your legs. This tells us that our brain can help our muscles get stronger. But we need more research to know if this works for people who already exercise a lot.",
     'relevance task': 'Use terms a 12-year-old can understand.',
     'preceding summary': '"New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights into the mind-muscle connection and its impact on muscular adaptations. Further research is needed to determine the effects of attentional focus strategies on individuals with previous 

In [223]:
def process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='simple'):
    """
    Merge the qna_dict and chatbot_dict into a single DataFrame. 
    Return the dataframe grouped by rows by qna_dict[iteration_id].columns
    """
    # Create an empty list to store the dataframes
    df_list = []
    # Iterate through the chatbot_dict and the simple_summary_dict
    for chatbot_key in chatbot_dict[iteration_id].keys():
        if results_type=='simple':
            results_dict = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict
        else:
            results_dict = chatbot_dict[iteration_id][chatbot_key].relevance_dict
        for iteration_key in results_dict.keys():
            for response_key in results_dict[iteration_key].keys():
                df_list.append(pd.DataFrame(results_dict[iteration_key][response_key]).transpose())
    
    # Concatenate the dataframes into a single dataframe
    new_results = pd.concat(df_list)
    
    # Merge the qna_dict and the simple_summaries dataframe
    new_results = qna_dict[iteration_id].merge(
        new_results, how='right', 
        right_on=f'{"original" if results_type=="simple" else "preceding"} summary', 
        left_on='summary'
    )
    print(f'{"simple summaries" if results_type=="simple" else "Added relevance"} dataframe shape:', new_results.shape)
    
    return new_results

process_chaining_results(qna_dict, chatbot_dict, iteration_id, results_type='relevance')

Added relevance dataframe shape: (8, 12)


,choice,article_title,system_role,model,prep step,summarization task,summary,text,preceding summary,relevance choice,relevance statement,relevance task
0,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"""New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights in...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","""New study suggests that an internal focus of attention is more effective than an external focus for maximizing hypertrophy in the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. These findings provide insights in...",1,"A new study found that when you focus on the muscles you're working on while lifting weights, it can help them grow bigger. But this only seems to work for the muscles in your arms, not your legs. This tells us that our brain can help our muscles get stronger. But we need more research to know i...",Use terms a 12-year-old can understand.
1,2,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,"A new study has found that an internal focus of attention is superior to an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study also...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","A new study has found that an internal focus of attention is superior to an external focus of attention when it comes to maximizing hypertrophy of the elbow flexors during long-term resistance training. However, attentional focus did not seem to affect lower extremity hypertrophy. The study also...",1,"A new study found that when you're lifting weights, it's better to focus on the muscles you're using inside your body rather than the things around you. This helps your arm muscles get bigger over time. But it doesn't seem to make a difference for your leg muscles. The study also showed that the...",Use terms a 12-year-old can understand.
2,1,Differential effects of attentional focus strategies during long-term resistance training,You are an expert at science communication.,gpt-3.5-turbo,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,"Hey, have you heard about the latest study on attentional focus strategies during long-term resistance training? It's really fascinating! The study found that an internal focus of attention resulted in superior hypertrophic increases in the elbow flexors compared to an external focus. This parti...","Differential effects of attentional focus strategies during long-term resistance training\n\nDISCUSSION \nThis is the first study to investigate the effects of different attentional focus strategies on long-term muscular adaptations. The study produced several novel and notable findings. First, ...","Hey, have

# *End of Page*